In [ ]:
# Find most relevant terms for each topic using KMeans clustering

In [ ]:
#!pip install --upgrade threadpoolctl
#!pip install wordcloud

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

In [ ]:
sample_documents = [
    """
    Economics focuses on the behaviour and interactions of economic agents and how economies work.
    Microeconomics analyzes what's viewed as basic elements in the economy, including individual agents
    and markets, their interactions, and the outcomes of interactions. Individual agents may include,
    for example, households, firms, buyers, and sellers. Macroeconomics analyzes the economy as a system
    where production, consumption, saving, and investment interact, and factors affecting it: employment
    of the resources of labour, capital, and land, currency inflation, economic growth,
    and public policies that have impact on these elements.
    """,
    """
    Literature is any collection of written work, but it is also used more narrowly for writings specifically
    considered to be an art form, especially prose fiction, drama, and poetry. In recent centuries, the
    definition has expanded to include oral literature, much of which has been transcribed. Literature is
    a method of recording, preserving, and transmitting knowledge and entertainment, and can also have a social,
    psychological, spiritual, or political role. Literature, as an art form, can also include works in
    various non-fiction genres, such as biography, diaries, memoir, letters, and the essay. Within its broad
    definition, literature includes non-fictional books, articles or other printed information on a particular
    subject.
    """

]

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import re
import nltk

In [ ]:
nltk.download('stopwords')
stop = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

In [ ]:
# get rid of stop words, non-alphanumeric text, lemmatize
def cleanDocument(text):
    words = text.split()

    cleaned = []
    for w in words:
        w = re.sub(r'\W+', '', w)
        #print(w)

        w = w.lower()
        w = lemmatizer.lemmatize(w)


        if w not in stop:
            cleaned.append(w)

    return ' '.join(cleaned)

In [ ]:
documents = [cleanDocument(text) for text in sample_documents]

In [ ]:
documents

In [ ]:
df_transcripts = pd.DataFrame({'text': documents})

In [ ]:
df_transcripts

In [ ]:
tfv = TfidfVectorizer(ngram_range = (1,1))

In [ ]:
vec_text = tfv.fit_transform(df_transcripts['text'])

In [ ]:
vec_text

In [ ]:
words = tfv.get_feature_names_out()

In [ ]:
words

In [ ]:
kmeans = KMeans(n_clusters = 2, n_init='auto')
kmeans.fit(vec_text)
cluster_words = kmeans.cluster_centers_
df_cluster_words = pd.DataFrame(cluster_words, columns=words).T

In [ ]:
df_cluster_words.iloc[0:10]

In [ ]:
df_cluster_words.sort_values(by=1, ascending=False)

In [ ]:
for i in range(0, 2):
    print('Document', i)
    print(df_cluster_words.sort_values(i, ascending=False)[i].head(10))
    print('\n')

In [ ]:
pred_text = """
The definition of literature broadly encompasses written works used to transmit culture. However,
literature is not always limited to the written word. The literature meaning can include stories
told in the oral tradition and visual literature, such as drama that is intended to be performed
before an audience.
"""

In [ ]:
X_pred = tfv.transform([cleanDocument(pred_text)])
kmeans.predict(X_pred)

In [ ]:
import numpy as np

# Get distances from each point to each centroid
distances = kmeans.transform(X_pred)  # shape (n_samples, n_clusters)

# Convert distances to similarity (smaller distance → higher similarity)
# Example: inverse-distance soft assignment
similarity = 1 / (distances + 1e-10)

# Normalize across clusters so they sum to 1 per sample
pseudo_proba = similarity / similarity.sum(axis=1, keepdims=True)


In [ ]:
pd.DataFrame(pseudo_proba, columns=[f"Cluster {i}" for i in range(kmeans.n_clusters)])